# Limpieza y Preparación de datos de adopciones
Este notebook contiene los pasos para inspeccionar, limpiar y preparar el dataset de adoptantes.

#### Índice del notebook: Limpieza y Preparación de Datos

##### 1. Inspección general del dataset
- 1.1. Información general del dataset  
- 1.2. Características de las columnas  
  - 1.2.1. Tipo de datos
  - 1.2.2. Valores nulos
  - 1.2.3. Valores únicos por columna  
  - 1.2.4. Valores duplicados
  - 1.2.5. Posibles errores ortográficos o inconsistencias de formato  
  - 1.2.6. Estadísticas y observaciones relevantes  

  
##### 2. Correcciones y transformación
- 2.1. Añadir un ID único  
- 2.2. Limpieza general de texto  
- 2.3. Limpieza específica  
  - 2.3.1. Limpieza de la columna 'fecha'
  - 2.3.2. Corregir fallos ortograficos en 'poblacion' & 'provincia'  
- 2.4. Clasificación y transformación de variables  
  - 2.4.1. Clasificar columna 'animal'  
  - 2.4.2. Agrupar 'tiempo_disponible' en rangos  
  - 2.4.3. Distribución por tipo de 'vivienda'  
  - 2.4.4. Agrupar rangos de 'edad'
  - 2.4.5. Distribución por 'horario_laboral'  
  - 2.4.6. Normalizar 'genero'  
  - 2.4.7. Distribución por 'experiencia_animales'  
  - 2.4.8. Normalizar variables de adopción 'devuelto'  
- 2.5. Revisión final  

  

##### 3. Guardado del dataset limpio

-------------------------------------------------------------------------------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# Ruta al archivo en Drive
ruta = '/content/drive/MyDrive/Dataset_TFM/Datos_Adopciones_Corregido_test.xlsx'

# Cargar el archivo
df = pd.read_excel(ruta)

Mounted at /content/drive


## 1.  Inspección general del dataset

### 1.1. Información general del dataset

In [ ]:
# Dimensiones del dataset
filas, columnas = df.shape
print(f"📏 El dataset tiene {filas} filas y {columnas} columnas.")

📏 El dataset tiene 2449 filas y 12 columnas.


In [ ]:
# Vista general del dataset
print("📌 Primeras filas del dataset:")
display(df.head())

📌 Primeras filas del dataset:


,animal,fecha,poblacion,provincia,nombre,edad,genero,vivienda,horario_laboral,experiencia_animales,devuelto,tiempo_disponible
0,Chia,2002-02-28 00:00:00,28019 Madrid,Madrid,Pedro,39,M,piso,jornada completa,baja,Adop,>5h
1,Rasputin,2002-03-04 00:00:00,28922 Alcorcon,Madrid,Sofía,45,F,piso,jornada parcial (mañana),baja,Adop,2-5h
2,Zeta,2012-11-24 00:00:00,Madrid,Madrid,Lucía,43,F,piso,jornada completa,baja,Adop,2-5h
3,Mia ( gata ),2012-12-06 00:00:00,Toledo,Toledo,Luis,20,M,piso,turno rotativo,media,Adop,2-5h
4,Agua,2012-12-14 00:00:00,Galapagos,Guadalajara,Lucía,49,F,casa/chalet,turno rotativo,baja,Adop,2-5h


### 1.2. Características de las columnas

#### 1.2.1. Tipo de datos

In [ ]:
df.dtypes.to_frame(name='tipo')

,tipo
animal,object
fecha,object
poblacion,object
provincia,object
nombre,object
edad,int64
genero,object
vivienda,object
horario_laboral,object
experiencia_animales,object


In [ ]:
# Tipos diferentes dentro de cada columna
for col in df.select_dtypes(include='object').columns:
    tipos = df[col].map(type).value_counts()
    if len(tipos) > 1:
        print(f"\n⚠️ Mezcla de tipos en '{col}':")
        print(tipos)


⚠️ Mezcla de tipos en 'animal':
animal
<class 'str'>      2447
<class 'float'>       2
Name: count, dtype: int64

⚠️ Mezcla de tipos en 'fecha':
fecha
<class 'datetime.datetime'>    2332
<class 'str'>                    80
<class 'int'>                    31
<class 'float'>                   6
Name: count, dtype: int64

⚠️ Mezcla de tipos en 'poblacion':
poblacion
<class 'str'>      2446
<class 'int'>         2
<class 'float'>       1
Name: count, dtype: int64

⚠️ Mezcla de tipos en 'genero':
genero
<class 'str'>      2436
<class 'float'>      13
Name: count, dtype: int64

⚠️ Mezcla de tipos en 'experiencia_animales':
experiencia_animales
<class 'str'>      2448
<class 'float'>       1
Name: count, dtype: int64


In [ ]:
# Convertir columna 'fecha' a datetime
df['fecha'] = pd.to_datetime(df['fecha'], errors='coerce', dayfirst=True)

#### 1.2.2. Valores nulos

In [ ]:
df.isnull().sum()

,0
animal,2
fecha,68
poblacion,1
provincia,0
nombre,0
edad,0
genero,13
vivienda,0
horario_laboral,0
experiencia_animales,1


In [ ]:
import numpy as np

# Reemplazar valores vacíos o solo con espacios por NaN en columnas de texto
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].apply(lambda x: np.nan if pd.isnull(x) or str(x).strip() == "" else x)

# Mostrar columnas con valores nulos
print("\n Valores nulos después de limpieza de strings vacíos:")
print(df.isnull().sum())


 Valores nulos después de limpieza de strings vacíos:
animal                   2
fecha                   68
poblacion                1
provincia                0
nombre                  13
edad                     0
genero                  13
vivienda                 0
horario_laboral          0
experiencia_animales     1
devuelto                16
tiempo_disponible        0
dtype: int64


#### 1.2.3. Valores únicos por columna

In [ ]:
df.nunique()

,0
animal,1758
fecha,1409
poblacion,733
provincia,85
nombre,382
edad,43
genero,7
vivienda,5
horario_laboral,7
experiencia_animales,3


#### Lo más llamativo:
- animal	1758	- muchos nombres distintos sin especificar el tipo de animal
- genero	7	- debería tener 2-3 valores
- vivienda	4	- conversión a category
- horario_laboral	5	- lo mismo que vivienda
- devuelto	8	- demasiados valores, debería ser sí/no

#### 1.2.4. Valores duplicados

In [ ]:
df.duplicated().sum()

np.int64(0)

No hay filas duplicadas.

In [ ]:
print("Nombres más repetidos:")
print(df['nombre'].value_counts().head(10))

print("\n Animales más repetidos:")
print(df['animal'].value_counts().head(10))

Nombres más repetidos:
nombre
Laura     209
Ana       206
Sofía     200
David     200
Lucía     199
Juan      196
Luis      194
Marta     192
Pedro     182
Carlos    178
Name: count, dtype: int64

 Animales más repetidos:
animal
Cachorros              27
No indica              14
Cachorro               14
Zoe                    14
Linda                  12
Portos                 12
Clara                  11
Rasputin (cachorro)    11
Olivia                 10
Copito                 10
Name: count, dtype: int64


#### 1.2.5. Posibles errores ortográficos o inconsistencias de formato

#### Revisión ortográfica reutilizable y corrección manual

Para detectar errores ortográficos o inconsistencias en columnas de texto (como `provincia` o `poblacion`), se creó una función reutilizable basada en `difflib.get_close_matches`. Esta herramienta permite encontrar entradas muy parecidas entre sí, lo cual puede indicar errores de escritura, abreviaciones o variaciones innecesarias.

In [ ]:
import re

def quitar_codigo_postal(valor):
        return re.sub(r'^\d{4,5}\s*', '', str(valor)).strip()

# Aplicar a la columna 'poblacion'
df['poblacion'] = df['poblacion'].apply(quitar_codigo_postal)

In [ ]:
from difflib import get_close_matches

def sugerir_correcciones(df, columna, cutoff=0.8):
    print(f"\n Posibles errores ortográficos en '{columna}':")
    valores = df[columna].dropna().astype(str).str.strip().unique()
    valores = sorted(valores)
    ya_mostrados = set()

    for val in valores:
        if val in ya_mostrados:
            continue
        similares = get_close_matches(val, valores, n=5, cutoff=cutoff)
        if len(similares) > 1:
            print("  -", ", ".join(similares))
            ya_mostrados.update(similares)

# Uso en columnas clave:
sugerir_correcciones(df, 'provincia', cutoff=0.8)
sugerir_correcciones(df, 'poblacion', cutoff=0.9)


 Posibles errores ortográficos en 'provincia':
  - ASturias, Asturias
  - Biskaia, Bizkaia
  - Bizcaia, Bizkaia
  - Canarias, Cantabria
  - Castellon, Castellón
  - Cataluña, Catañuña
  - Gerona, Girona
  - MAdrid, Madrid
  - Madrir, Madrid
  - No indica, no indica
  - Pais Vasco, Pais Vazco
  - Palencia, Valencia
  - Pontevedra, Pontevedra)
  - S.C.Tenerife, Tenerife
  - Toleddo, Toledo
  - ValenciA, Valencia

 Posibles errores ortográficos en 'poblacion':
  - ,Madrid, Madrid
  - Alcala de Henares, Alcalá de Henares
  - Arcicollar, Arcicóllar
  - Arganda del Rey, Arganda del rey
  - Becerril de La Sierra, Becerril de la Sierra
  - Boadilla del Monte, Boadilla del monte
  - Cedillo del Condado, Cedillo del condado
  - Collado Villalba, Collado Villlalba
  - El Viso San Juan, El Viso de San Juan
  - La Puebla de Almoradiel, Puebla de Almoradiel
  - La Torre de Esteban Hambran, La torre de Esteban Hambran
  - Las Ventas de Retamosa, Ventas de Retamosa
  - Maddrid, Madrid
  - Numancia de

In [ ]:
df['genero'].value_counts(dropna=False)

,count
genero,
F,1018
M,957
f,243
m,93
otro,66
mujer,42
hombre,17
NaN,13


In [ ]:
df['devuelto'].value_counts(dropna=False)

,count
devuelto,
Adop,928
No,907
adop,356
Sí,205
x,21
NaN,16
Dev,15
X,1


#### 1.2.6. Estadísticas y observaciones relevantes

In [ ]:
# Redondear resultados
print("Edad mínima:", int(df['edad'].min()))
print("Edad máxima:", int(df['edad'].max()))
print("Edad media:", round(df['edad'].mean()))
print("Edad mediana:", int(df['edad'].median()))
print("Desviación estándar:", round(df['edad'].std()))
print("Moda:", int(df['edad'].mode()[0]))

Edad mínima: 18
Edad máxima: 60
Edad media: 36
Edad mediana: 36
Desviación estándar: 11
Moda: 41


In [ ]:
cols_interes = ['animal', 'genero', 'vivienda', 'horario_laboral', 'experiencia_animales', 'devuelto']
display(df[cols_interes].describe())

,animal,genero,vivienda,horario_laboral,experiencia_animales,devuelto
count,2447,2436,2449,2449,2448,2433
unique,1758,7,5,7,3,7
top,Cachorros,F,piso,jornada completa,media,Adop
freq,27,1018,1406,1452,1046,928


## 2. Correcciones y transformación

### 2.1. Añadir un ID único

In [ ]:
# Añadir ID

df['ID'] = range(1, len(df) + 1)

# Reordenar columnas
cols = ['ID'] + [col for col in df.columns if col != 'ID']
df = df[cols]

In [ ]:
df.head()

,ID,animal,fecha,poblacion,provincia,nombre,edad,genero,vivienda,horario_laboral,experiencia_animales,devuelto,tiempo_disponible
0,1,Chia,2002-02-28,Madrid,Madrid,Pedro,39,M,piso,jornada completa,baja,Adop,>5h
1,2,Rasputin,2002-03-04,Alcorcon,Madrid,Sofía,45,F,piso,jornada parcial (mañana),baja,Adop,2-5h
2,3,Zeta,2012-11-24,Madrid,Madrid,Lucía,43,F,piso,jornada completa,baja,Adop,2-5h
3,4,Mia ( gata ),2012-12-06,Toledo,Toledo,Luis,20,M,piso,turno rotativo,media,Adop,2-5h
4,5,Agua,2012-12-14,Galapagos,Guadalajara,Lucía,49,F,casa/chalet,turno rotativo,baja,Adop,2-5h


### 2.2. Limpieza general de texto

- Eliminar tildes y caracteres especiales
- Reemplazar saltos de línea por espacios
- Normalizar capitalización


In [ ]:
import unicodedata
import re

def limpiar_texto_completo(texto):
    if pd.isnull(texto):
        return texto
    texto = str(texto)
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    texto = texto.replace('\n', ' ').replace('\r', ' ')
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto.title()

columnas_a_normalizar = ['animal', 'provincia', 'poblacion', 'nombre', 'vivienda', 'horario_laboral', 'experiencia_animales', 'devuelto', 'tiempo_disponible']

for col in columnas_a_normalizar:
    df[col] = df[col].apply(limpiar_texto_completo)

In [ ]:
df.head()

,ID,animal,fecha,poblacion,provincia,nombre,edad,genero,vivienda,horario_laboral,experiencia_animales,devuelto,tiempo_disponible
0,1,Chia,2002-02-28,Madrid,Madrid,Pedro,39,M,Piso,Jornada Completa,Baja,Adop,>5H
1,2,Rasputin,2002-03-04,Alcorcon,Madrid,Sofia,45,F,Piso,Jornada Parcial (Manana),Baja,Adop,2-5H
2,3,Zeta,2012-11-24,Madrid,Madrid,Lucia,43,F,Piso,Jornada Completa,Baja,Adop,2-5H
3,4,Mia ( Gata ),2012-12-06,Toledo,Toledo,Luis,20,M,Piso,Turno Rotativo,Media,Adop,2-5H
4,5,Agua,2012-12-14,Galapagos,Guadalajara,Lucia,49,F,Casa/Chalet,Turno Rotativo,Baja,Adop,2-5H


### 2.3. Limpieza específica


#### 2.3.1. Limpieza de la columna fecha

In [ ]:
# Extraer el año
df['año'] = df['fecha'].dt.year

# Eliminar filas con año nulo
df = df[df['año'].notna()]

# Convertir 'año' a entero
df['año'] = df['año'].astype(int)

# Conteo por año
print("📅 Conteo de registros por año (sin filtrar):")
print(df['año'].value_counts().sort_index())

📅 Conteo de registros por año (sin filtrar):
año
1905      9
1970     31
2002      2
2004      1
2006      2
2008      2
2009      4
2010     25
2011     29
2012     56
2013    218
2014    215
2015    213
2016    205
2017    120
2018    111
2019     94
2020     49
2021    332
2022    281
2023    259
2024    105
2025     17
2107      1
Name: count, dtype: int64


Se observa que hay registros con años claramente fuera del rango esperable (ej. 1905, 2107), lo cual sugiere errores en la entrada de datos. Por eso se decide conservar solo registros entre 2010 y 2025 para asegurar relevancia y coherencia en el análisis.


In [ ]:
# Crear columna para visualización
df['fecha_europea'] = df['fecha'].dt.strftime('%d/%m/%Y')

# Convertir a date (sin hora)
df['fecha'] = df['fecha'].dt.date

# Eliminar nulos y filtrar rango esperado
df = df[df['año'].notna()]
df = df[df['año'].between(2010, 2025)]

In [ ]:
df[['fecha', 'fecha_europea', 'año']].head()

,fecha,fecha_europea,año
2,2012-11-24,24/11/2012,2012
3,2012-12-06,06/12/2012,2012
4,2012-12-14,14/12/2012,2012
5,2012-12-20,20/12/2012,2012
6,2012-12-22,22/12/2012,2012


### 2.3.2. Corregir fallos ortograficos en 'poblacion' & 'provincia'

#### Corrección de errores ortográficos en 'poblacion' y 'provincia'

Tras normalizar el texto y eliminar códigos postales, aún pueden quedar inconsistencias ortográficas en columnas como 'poblacion' y 'provincia'. Se utiliza una función basada en `difflib` para sugerir valores similares que podrían representar errores de escritura. Esto permite identificar grupos de valores equivalentes y proceder a una corrección manual en los casos más evidentes.

In [ ]:
# Limpiar números al principio de 'provincia' y 'poblacion'
for col in ['provincia', 'poblacion']:
    df[col] = df[col].astype(str).str.replace(r'^\d+\s*', '', regex=True).str.strip()

In [ ]:
df = df[df['poblacion'].notna() & (df['poblacion'].str.strip() != '')]

In [ ]:
import pandas as pd

# Mostrar valores únicos antes de limpieza
print("Provincias únicas ANTES de limpiar:")
print(sorted(df['provincia'].dropna().unique()))
print("\nPoblaciones únicas ANTES de limpiar:")
print(sorted(df['poblacion'].dropna().astype(str).unique()))

# Diccionarios de corrección
correcciones_provincia = {
    'ASturias' : 'Asturias',
    'Madrir': 'Madrid',
    'MAdrid': 'Madrid',
    'Toleddo': 'Toledo',
    'Bizcaya': 'Bizkaia',
    'Biskaia': 'Bizkaia',
    'Bizcaia': 'Bizkaia',
    'Belgica': 'Bélgica',
    'Alemania': 'Alemania',
    'Luxemburgo': 'Luxemburgo',
    'no indica': 'Sin datos',
    'No indica': 'Sin datos',
    '08758 Barcelona': 'Barcelona',
    'Catañuña': 'Cataluña',
    'ValenciA': 'Valencia',
    'Castellón': 'Castellon',
    'Pontevedra)': 'Pontevedra'
}

correcciones_poblacion = {
    ', Cerdanyola Del Valles,' : 'Cerdanyola Del Valles',
    'Collado Villlalba': 'Collado Villalba',
    'S.C.Retamar': 'Santa Cruz Retamar',
    'S.Cugat Del Valles': 'Sant Cugat Del Valles',
    'La Puebla De Almoradiel': 'Puebla De Almoradiel',
    'Rivas Vaciamdadrid': 'Rivas Vaciamadrid',
    'San Rafaelk': 'San Rafael',
    'Maddrid': 'Madrid',
    ',Madrid' : 'Madrid',
    'S.F.Henares': 'San Fernando De Henares',
    'S.F. Henares': 'San Fernando De Henares',
    'S.S. Reyes': 'San Sebastian De Los Reyes',
    'S.S.Reyes': 'San Sebastian De Los Reyes',
    'S.S.De Los Reyes': 'San Sebastian De Los Reyes',
    'S.L.Escorial': 'San Lorenzo De El Escorial',
    'S.L.El Escorial': 'San Lorenzo De El Escorial',
    'S.Esteban De Palautordera': 'Sant Esteve De Palautordera',
    'Anover De Tajo ': 'Anover De Tajo',
}

# Aplicar correcciones
df['provincia'] = df['provincia'].replace(correcciones_provincia)
df['poblacion'] = df['poblacion'].replace(correcciones_poblacion)

# Mostrar valores únicos después de limpieza
print("\nProvincias únicas DESPUÉS de limpiar:")
print(sorted(df['provincia'].dropna().unique()))
print("\nPoblaciones únicas DESPUÉS de limpiar:")
print(sorted(df['poblacion'].dropna().astype(str).unique()))

Provincias únicas ANTES de limpiar:
['Alava', 'Albacete', 'Alemania', 'Alicante', 'Almeria', 'Andalucia', 'Asturias', 'Avila', 'Badajoz', 'Badalona', 'Baleares', 'Barcelona', 'Belgica', 'Bilbao', 'Biskaia', 'Bizcaia', 'Bizkaia', 'Burgos', 'Caceres', 'Cadiz', 'Canarias', 'Cantabria', 'Castellon', 'Castilla La Mancha', 'Cataluna', 'Catanuna', 'Ciudad Real', 'Cordoba', 'Cuenca', 'Dinamarca', 'Extremadura', 'Francia', 'Galicia', 'Gerona', 'Girona', 'Granada', 'Guadalajara', 'Guipuzcoa', 'Huelva', 'Huesca', 'Jaen', 'La Coruna', 'La Rioja', 'Leon', 'Luxemburgo', 'Madrid', 'Madrir', 'Malaga', 'Mallorca', 'Murcia', 'Navarra', 'No Indica', 'Olias Del Rey', 'Pais Vasco', 'Pais Vazco', 'Palencia', 'Pamplona', 'Pontevedra', 'Pontevedra)', 'Portugal', 'Reino Unido', 'Rioja', 'S.C.Tenerife', 'Salamanca', 'San Sebastian', 'Segovia', 'Sin Datos', 'Soria', 'Suiza', 'Tarragona', 'Tenerife', 'Toleddo', 'Toledo', 'Valencia', 'Valladolid', 'Valparaiso', 'Vizcaya', 'Zamora', 'Zaragoza']

Poblaciones únicas 

### 2.4. Clasificación y transformación de variables

#### 2.4.1. Clasificar columna animal

In [ ]:
import re
import pandas as pd

PARENTESIS = re.compile(r"\(([^)]*)\)")

def clasificar_animal(nombre):
    if pd.isna(nombre) or str(nombre).strip() == "":
        return "perro"

    texto = str(nombre)

    # Buscar dentro de paréntesis
    for contenido in PARENTESIS.findall(texto):
        cont = contenido.lower().strip()

        # gato/gata/gatito/gatita
        if re.search(r"\b(gato|gata|gatito|gatita)\b", cont, flags=re.I):
            return "gato"

    return "perro"

# Aplicar al DataFrame
df["tipo_animal"] = df["animal"].apply(clasificar_animal)

# Chequeo rápido
print(df["tipo_animal"].value_counts())

tipo_animal
perro    2180
gato      147
Name: count, dtype: int64


In [ ]:
# Reordenar columnas
cols = df.columns.tolist()
cols.insert(cols.index('animal') + 1, cols.pop(cols.index('tipo_animal')))
df = df[cols]
df.head()

,ID,animal,tipo_animal,fecha,poblacion,provincia,nombre,edad,genero,vivienda,horario_laboral,experiencia_animales,devuelto,tiempo_disponible,año,fecha_europea
2,3,Zeta,perro,2012-11-24,Madrid,Madrid,Lucia,43,F,Piso,Jornada Completa,Baja,Adop,2-5H,2012,24/11/2012
3,4,Mia ( Gata ),gato,2012-12-06,Toledo,Toledo,Luis,20,M,Piso,Turno Rotativo,Media,Adop,2-5H,2012,06/12/2012
4,5,Agua,perro,2012-12-14,Galapagos,Guadalajara,Lucia,49,F,Casa/Chalet,Turno Rotativo,Baja,Adop,2-5H,2012,14/12/2012
5,6,Beda (Chispa ),perro,2012-12-20,Madrid,Madrid,Luis,20,M,Atico,Jornada Completa,Baja,No,2-5H,2012,20/12/2012
6,7,Bimbo,perro,2012-12-22,Vigo,Pontevedra,Laura,19,F,Casa/Chalet,Jornada Parcial (Tarde),Alta,No,2-5H,2012,22/12/2012


#### 2.4.2. Agrupar tiempo_disponible en rangos

In [ ]:
mapeo = {
    '<2h': '<1–2h',
    '2-5h': '2–5h',
    '>5h': '>5h'
}

# Aplicar mapeo directamente sobre la columna original
df['tiempo_disponible'] = df['tiempo_disponible'].str.strip().str.lower().map(mapeo)

In [ ]:
print(df['tiempo_disponible'].value_counts())

tiempo_disponible
2–5h     1643
>5h       464
<1–2h     220
Name: count, dtype: int64


#### 2.4.3. Distribución por tipo de vivienda





In [ ]:
# Conteo con frecuencia y porcentaje
vivienda_counts = df['vivienda'].value_counts(dropna=False)
vivienda_percent = df['vivienda'].value_counts(normalize=True, dropna=False) * 100

vivienda_summary = pd.DataFrame({
    'Frecuencia': vivienda_counts,
    'Porcentaje (%)': vivienda_percent.round(2)
})

display(vivienda_summary)

,Frecuencia,Porcentaje (%)
vivienda,,
Piso,1344,57.76
Casa/Chalet,554,23.81
Atico,251,10.79
Vivienda Compartida,177,7.61
Piso Compartido,1,0.04


In [ ]:
# Sustituir 'Vivienda Compartida' y 'Piso compartido' en la columna original
df['vivienda'] = df['vivienda'].replace({
    'Vivienda Compartida': 'Vivienda compartida',
    'Piso compartido': 'Vivienda compartida',
    'Piso Compartido': 'Vivienda compartida'
})

# Calcular las frecuencias y porcentajes de la columna ya unificada
vivienda_counts = df['vivienda'].value_counts(dropna=False)
vivienda_percent = df['vivienda'].value_counts(normalize=True, dropna=False) * 100

# Construir el DataFrame resumen
vivienda_summary = pd.DataFrame({
    'Frecuencia': vivienda_counts,
    'Porcentaje (%)': vivienda_percent.round(2)
})

display(vivienda_summary)

,Frecuencia,Porcentaje (%)
vivienda,,
Piso,1344,57.76
Casa/Chalet,554,23.81
Atico,251,10.79
Vivienda compartida,178,7.65


### 2.4.4. Agrupar rangos de edad

In [ ]:
# Columna 'edad' en formato numérico
df.loc[:, "edad"] = pd.to_numeric(df["edad"], errors="coerce")

# Crear rangos de edad
bins = [17, 24, 34, 44, 54, 64, float("inf")]
labels = ["18–24", "25–34", "35–44", "45–54", "55–64", "65+"]

# Asignar cada edad a un grupo
df.loc[:,"grupo_edad"] = pd.cut(df["edad"], bins=bins, labels=labels, right=True, include_lowest=True)

df['grupo_edad'].value_counts().sort_index()

,count
grupo_edad,
18–24,363
25–34,734
35–44,705
45–54,340
55–64,185
65+,0


#### 2.4.5. Distribución por horario laboral

In [ ]:
# Conteo de categorías en variables clave
columnas = ['horario_laboral']

for col in columnas:
    print(f"\n📊 Distribución de '{col}':")
    print(df[col].value_counts(dropna=False))


📊 Distribución de 'horario_laboral':
horario_laboral
Jornada Completa            1381
Turno Rotativo               324
Jornada Parcial (Tarde)      263
Jornada Parcial (Manana)     225
Sin Empleo                   128
Jornada Rotativa               3
Jornada Parcial                3
Name: count, dtype: int64


#### 2.4.6. Normalizar genero

In [ ]:
def normalizar_genero(valor):
    valor = str(valor).strip().lower()

    if valor in ['m', 'masculino', 'hombre']:
        return 'Hombre'
    elif valor in ['f', 'femenino', 'mujer']:
        return 'Mujer'
    else:
        return 'Otro'

df.loc[:, 'genero'] = df['genero'].apply(normalizar_genero)

In [ ]:
df['genero'].value_counts()

,count
genero,
Mujer,1227
Hombre,1044
Otro,56


In [ ]:
df[df['genero'] == 'Otro']

,ID,animal,tipo_animal,fecha,poblacion,provincia,nombre,edad,genero,vivienda,horario_laboral,experiencia_animales,devuelto,tiempo_disponible,año,fecha_europea,grupo_edad
2016,2017,Chay,perro,2011-01-12,Alcala De Henares,Madrid,NaN,30,Otro,Piso,Jornada Completa,Media,X,2–5h,2011,12/01/2011,25–34
2017,2018,Cuca,perro,2010-01-05,Fuenlabrada,Madrid,NaN,27,Otro,Piso,Jornada Parcial (Manana),Alta,X,>5h,2010,05/01/2010,25–34
2020,2021,Pipin,perro,2012-01-02,Alcorcon,Madrid,NaN,38,Otro,Piso,Jornada Completa,Media,X,>5h,2012,02/01/2012,35–44
2021,2022,Silver,perro,2011-01-09,Mostoles,Madrid,NaN,53,Otro,Piso,Jornada Completa,Baja,X,<1–2h,2011,09/01/2011,45–54
2027,2028,Sombra,perro,2013-09-01,Toledo,Toledo,Apao,56,Otro,Vivienda compartida,Jornada Completa,Alta,X,2–5h,2013,01/09/2013,55–64
2028,2029,Romi,perro,2012-09-01,Sesena,Toledo,Apao,56,Otro,Piso,Jornada Completa,Media,X,>5h,2012,01/09/2012,55–64
2032,2033,Moon,perro,2011-01-01,Alcorcon,Madrid,Torre,37,Otro,Piso,Jornada Parcial (Tarde),Media,Adop,>5h,2011,01/01/2011,35–44
2036,2037,Zazu,perro,2013-07-22,Sesena,Toledo,NaN,21,Otro,Casa/Chalet,Jornada Parcial (Manana),Baja,X,>5h,2013,22/07/2013,18–24
2041,2042,Vilma,perro,2013-01-08,Illescas,Toledo,Gaci,24,Otro,Piso,Jornada Completa,Media,Si,>5h,2013,08/01/2013,18–24
2042,2043,Alfil,perro,2014-01-05,Illescas,Toledo,Levrier Italia - Gallia,27,Otro,Vivienda compartida,Jornada Parcial (Tarde),Media,NaN,>5h,2014,05/01/2014,25–34


Nota: La categoría "Otro" suele representar cesiones a otras protectoras o filas sin nombre del adoptante.

#### 2.4.7. Distribución por experiencia_animales

In [ ]:
df['experiencia_animales'].value_counts()

,count
experiencia_animales,
Media,958
Alta,855
Baja,513


#### 2.4.8. Normalizar variables de adopción (devuelto)

In [ ]:
def normalizar_devuelto(valor):
    valor = str(valor).strip().lower()
    if valor in ['dev', 'sí', 'si']:
        return 'si'
    elif valor in ['adop', 'no']:
        return 'no'
    else:
        return 'otro'

# Aplicar al dataframe
df.loc[:,'devuelto'] = df['devuelto'].apply(normalizar_devuelto)

In [ ]:
df['devuelto'].value_counts()

,count
devuelto,
no,2091
si,211
otro,25


In [ ]:
df[df['devuelto'] == 'otro']

,ID,animal,tipo_animal,fecha,poblacion,provincia,nombre,edad,genero,vivienda,horario_laboral,experiencia_animales,devuelto,tiempo_disponible,año,fecha_europea,grupo_edad
2016,2017,Chay,perro,2011-01-12,Alcala De Henares,Madrid,NaN,30,Otro,Piso,Jornada Completa,Media,otro,2–5h,2011,12/01/2011,25–34
2017,2018,Cuca,perro,2010-01-05,Fuenlabrada,Madrid,NaN,27,Otro,Piso,Jornada Parcial (Manana),Alta,otro,>5h,2010,05/01/2010,25–34
2019,2020,Nuka,perro,2011-01-10,Leganes,Madrid,A - Teresa San Jose,19,Mujer,Piso,Jornada Parcial (Manana),Media,otro,2–5h,2011,10/01/2011,18–24
2020,2021,Pipin,perro,2012-01-02,Alcorcon,Madrid,NaN,38,Otro,Piso,Jornada Completa,Media,otro,>5h,2012,02/01/2012,35–44
2021,2022,Silver,perro,2011-01-09,Mostoles,Madrid,NaN,53,Otro,Piso,Jornada Completa,Baja,otro,<1–2h,2011,09/01/2011,45–54
2023,2024,Sultan,perro,2010-01-04,Getafe,Madrid,Tamara Y Ruth,51,Mujer,Casa/Chalet,Turno Rotativo,Media,otro,2–5h,2010,04/01/2010,45–54
2024,2025,Venus M,perro,2013-04-01,Alcorcon,Madrid,A- Pilar,20,Mujer,Piso,Sin Empleo,Media,otro,2–5h,2013,01/04/2013,18–24
2025,2026,Vito,perro,2013-04-01,Mostoles,Madrid,Graci,58,Mujer,Casa/Chalet,Jornada Completa,Alta,otro,2–5h,2013,01/04/2013,55–64
2027,2028,Sombra,perro,2013-09-01,Toledo,Toledo,Apao,56,Otro,Vivienda compartida,Jornada Completa,Alta,otro,2–5h,2013,01/09/2013,55–64
2028,2029,Romi,perro,2012-09-01,Sesena,Toledo,Apao,56,Otro,Piso,Jornada Completa,Media,otro,>5h,2012,01/09/2012,55–64


Se normalizó la columna `devuelto` agrupando respuestas como "sí", "dev" y "si" bajo `sí`, y "no", "adop" bajo `no`. Las respuestas no clasificables se etiquetan como `otro`, y pueden excluirse.

### 2.5. Revisión final

In [ ]:
# Verificar si quedan valores nulos
print(df.isnull().sum())

ID                      0
animal                  0
tipo_animal             0
fecha                   0
poblacion               0
provincia               0
nombre                  8
edad                    0
genero                  0
vivienda                0
horario_laboral         0
experiencia_animales    1
devuelto                0
tiempo_disponible       0
año                     0
fecha_europea           0
grupo_edad              0
dtype: int64


In [ ]:
# Normalizar la columna a texto para buscar '0' y valores vacíos en 'nombre'
col_exp = df['nombre'].astype(str).str.strip()

# Filtrar filas (NaN/""/None)
filas_exp_cero = df[(col_exp == '0') | (df['nombre'].isna())]

print("Filas con valor 0 o nulo en nombre:", len(filas_exp_cero))
print(filas_exp_cero[['animal','tipo_animal', 'nombre', 'experiencia_animales', 'genero', 'devuelto']])

Filas con valor 0 o nulo en nombre: 8
            animal tipo_animal nombre experiencia_animales genero devuelto
2016          Chay       perro    NaN                Media   Otro     otro
2017          Cuca       perro    NaN                 Alta   Otro     otro
2020         Pipin       perro    NaN                Media   Otro     otro
2021        Silver       perro    NaN                 Baja   Otro     otro
2036          Zazu       perro    NaN                 Baja   Otro     otro
2108  Okupa / Milu       perro    NaN                 Baja   Otro       si
2360        Halley       perro    NaN                 Alta   Otro       no
2382          Runi       perro    NaN                 Alta   Otro       no


In [ ]:
# Eliminar filas con valores nulos en la columna 'nombre'
df = df.dropna(subset=['nombre']).reset_index(drop=True)

In [ ]:
# Normalizar la columna a texto para buscar '0' y valores vacíos en 'experiencia_animales'
col_exp = df['experiencia_animales'].astype(str).str.strip()

# Filtrar filas (NaN/""/None)
filas_exp_cero = df[(col_exp == '0') | (df['experiencia_animales'].isna())]

print("Filas con valor 0 o nulo en experiencia_animales:", len(filas_exp_cero))
print(filas_exp_cero[['animal', 'experiencia_animales', 'genero', 'devuelto']])

Filas con valor 0 o nulo en experiencia_animales: 1
              animal experiencia_animales genero devuelto
1999  Marieta (Gata)                  NaN  Mujer       no


In [ ]:
# Sustituir los valores NaN en experiencia_animales por 'baja'
df['experiencia_animales'] = df['experiencia_animales'].fillna('baja')

Las filas en las que el campo 'nombre' aparece vacío se han eliminado del análisis, ya que sin el nombre del adoptante no es posible inferir con fiabilidad el género, variable que considero clave en este estudio. Además, estos registros representan únicamente 8 casos, por lo que su exclusión no afecta de manera significativa a los resultados finales.

In [ ]:
# Verificar si quedan valores nulos
print(df.isnull().sum())

ID                      0
animal                  0
tipo_animal             0
fecha                   0
poblacion               0
provincia               0
nombre                  0
edad                    0
genero                  0
vivienda                0
horario_laboral         0
experiencia_animales    0
devuelto                0
tiempo_disponible       0
año                     0
fecha_europea           0
grupo_edad              0
dtype: int64


## 3. Guardar dataset limpio

In [ ]:
# Guardar dataset limpio
ruta_guardado = "/content/drive/MyDrive/Dataset_TFM/datos_adopciones_limpio_test.xlsx"

# Exportar a Excel
df.to_excel(ruta_guardado, index=False)

print(f"Dataset guardado correctamente en: {ruta_guardado}")

Dataset guardado correctamente en: /content/drive/MyDrive/Dataset_TFM/datos_adopciones_limpio_test.xlsx
